In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
# dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# loading your own data (only on google colab)
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [4]:
# Read and decode the file
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of the text: {} characters'.format(len(text)))

Length of the text: 1115394 characters


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# Encoding
vocab = sorted(set(text))   # saves only unique characters because it's set and can't have duplicates

char2idx = {u: i for i, u in enumerate(vocab)} # make a dict of words that return a number
idx2char = np.array(vocab)                     # make a numpy array that returns a work with given integer

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
print(len(text_as_int))

1115394


In [88]:
print("Text:", text[:13])
print("Encoded", text_to_int(text[:13]))

Text: First Citizen
Encoded [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [89]:
# convert numberic value to text
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [90]:
# Creating training examples
seq_length = 100        # length of sequence for a training example
examples_per_epochs = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [91]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [92]:
# Split into input and output
def split_input_target(chunk):
    input_text = chunk[:-1]     # hell
    target_text = chunk[1:]     # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)

In [93]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [94]:
# Make training batches
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)     # number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [95]:
# Building the model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),     # 64 entries and unknown sequence length
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)   # all chars get a probability of being written
    ])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_6 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_6 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [96]:
# Creating a loss function

In [97]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)      # ask untrained model for a prediction on ouly the first batch of training data
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [98]:
# prediction is an array of 64 arrays (a tensor), one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 7.77695968e-04  6.39738224e-04  2.03169463e-03 ... -1.89285493e-05
   -1.54321711e-03 -6.05596974e-03]
  [-8.45483039e-04  1.07012864e-03 -3.36983101e-03 ... -6.74847933e-03
   -2.01403117e-03 -2.45625083e-03]
  [-1.26469228e-03  1.54576194e-03 -1.04580971e-03 ... -1.81564060e-03
   -2.36053276e-03  1.66473945e-03]
  ...
  [-3.41779739e-03 -7.40789808e-03 -5.19507006e-03 ... -1.76967867e-03
    8.51488207e-03 -3.68018099e-03]
  [-1.38771138e-03 -4.85506980e-03 -2.54219258e-03 ...  3.75598553e-04
    4.44052042e-03 -1.01981899e-02]
  [ 3.45741352e-03  8.05082615e-04  2.27783108e-04 ...  1.22378115e-03
   -2.73963762e-03 -6.89533213e-03]]

 [[-3.96347605e-05  6.14016084e-03  1.82525895e-03 ...  1.69300579e-03
    3.04102851e-03  4.15699463e-03]
  [ 1.69914938e-03  3.37603735e-03  2.83990381e-03 ...  3.43322032e-03
   -5.48522454e-04  3.64905503e-03]
  [ 1.09238643e-03  2.78782658e-03 -4.46950505e-03 ... -5.73534332e-03
   -2.26590689e-03  6.20287471e-03]
  ...
  [-4.060

In [99]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 7.7769597e-04  6.3973822e-04  2.0316946e-03 ... -1.8928549e-05
  -1.5432171e-03 -6.0559697e-03]
 [-8.4548304e-04  1.0701286e-03 -3.3698310e-03 ... -6.7484793e-03
  -2.0140312e-03 -2.4562508e-03]
 [-1.2646923e-03  1.5457619e-03 -1.0458097e-03 ... -1.8156406e-03
  -2.3605328e-03  1.6647395e-03]
 ...
 [-3.4177974e-03 -7.4078981e-03 -5.1950701e-03 ... -1.7696787e-03
   8.5148821e-03 -3.6801810e-03]
 [-1.3877114e-03 -4.8550698e-03 -2.5421926e-03 ...  3.7559855e-04
   4.4405204e-03 -1.0198190e-02]
 [ 3.4574135e-03  8.0508261e-04  2.2778311e-04 ...  1.2237811e-03
  -2.7396376e-03 -6.8953321e-03]], shape=(100, 65), dtype=float32)


In [100]:
# finally we look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 7.7769597e-04  6.3973822e-04  2.0316946e-03 -1.1612432e-02
 -3.7100972e-03  2.8371441e-03 -4.1937972e-03  2.2567075e-04
 -4.8389155e-03 -5.4234769e-03  2.1775011e-03 -5.0175041e-03
 -2.8521391e-03 -2.6248413e-04  2.6501841e-03 -2.0666046e-03
  2.7472714e-03  2.3458211e-03 -4.8187417e-03 -8.7728025e-05
  3.5929581e-05 -2.8522322e-03 -7.3567466e-03  2.0985084e-04
  2.1653438e-03  6.5829963e-03  1.2334986e-03 -2.7452749e-03
 -1.3528244e-03 -6.4258225e-04 -2.6518479e-03 -3.5489060e-03
 -1.1602850e-03 -8.2205544e-04 -8.7511633e-04  1.4177201e-03
  1.6514213e-03 -6.8266492e-04 -1.4768521e-03  2.7670744e-03
  2.3668727e-03 -3.0195098e-03 -5.3482191e-03  3.2652649e-03
 -4.9783257e-03 -2.3275407e-03 -4.6913689e-03 -5.6165177e-03
  8.1272540e-04 -1.0322529e-03  2.8428598e-03  9.7590196e-04
 -2.5865491e-03  5.4621941e-04 -2.1409472e-03 -1.9982690e-04
  5.3626411e-03 -1.9531995e-03  2.2694974e-03 -2.7325761e-04
 -5.0106095e-03 -9.4954460e-04 -1.8928549e-05 -1.5432171e-03
 -6.055969

In [101]:
# Determine the predicted character
sampled_indices = tf.random.categorical(pred, num_samples=1) # sampling the probability instead of picking max value from prediction (GOOGLE IT!!)

# reshape array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars     # what model predicted

"zmGrM&szBYScZ.sLi.f-AGzOYE,qSRwF-PNwyp3DuhVyY.;Ibt:vvA,?eHDpaUHDOhqUPQGM.ouvxt?I\nmJ3KzO'A,Pu,,FVzjFT"

In [102]:
# LOSS FUNCTION
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [103]:
# Compiling the model
model.compile(optimizer='adam', loss=loss)

In [104]:
# Creating checkpoints in dir
checkpoint_dir = "./training_checkpoints"
# Name of checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [105]:
# Training the model
history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

172/172 [==============================] - 31s 166ms/step - loss: 2.6678


In [106]:
# Loading the model
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [107]:
# Load latest checkpoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [108]:
# Load specific checkpoint
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

In [109]:
# Generating text
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [112]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

romeon, apance on os lit
Thes houls'sen, I ary thou no'ce base
hom hentoless,
e buteagf'd. Is for Whatpey,
Mowid. hull's flevebpeteas if so mer.
And I te mart sy lath opcinput.
I
LPRSHARA:
By yow sle gove.

GesGrof:
Te dige thass boncelll be nis?
Fret young.

FiETt:
So bwallo Gaverecith;
And of wath limid y ue to thourrer,
Crovaread sroul wor and bucond'sw andt. I andy
Tpronty me deem:
Urdor linf Esather theneds, wort rof ay;
her of sreainst dad e ceakidn me,

VARDLiS:
Mir i, intherseath, we pard froua's gravedt
Cathin sace wedeed he youthurs
Mather: my hackith ghegtlowss
OPlved:
I amen't:
Yey warld efarothirs fuesser:
And hamt, is apearbde.
Whan low': alid; FUny yot lith vee hlveld brcithet:
Bunseer meele.
Whyous
Ond an mo crouncy'sed bred,
Wheill'bbsand lids, bode, of spatyt,
Hour, wingequry 
